<a href="https://colab.research.google.com/github/VinayakG2002/NLPlay-with-Transformers/blob/main/RoBERTa_small_database.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install transformers

     |████████████████████████████████| 2.5MB 8.2MB/s 
     |████████████████████████████████| 3.3MB 49.8MB/s 
     |████████████████████████████████| 901kB 37.8MB/s 


In [2]:
pip install datasets

     |████████████████████████████████| 266kB 9.3MB/s 
     |████████████████████████████████| 122kB 55.4MB/s 
     |████████████████████████████████| 245kB 49.5MB/s 


In [3]:
import pandas as pd
import os
import torch
import numpy as np
import nltk
import matplotlib.pyplot as plt
import random

In [4]:
url='https://raw.githubusercontent.com/VinayakG2002/NLPlay-with-Transformers/main/IMDB%20Dataset.csv?token=ATKA2LIXPHWLGQT64HBC2KLA6XEPW'
df = pd.read_csv(url)

In [5]:
def remove_tags(text):
    final = ""
    stt = True
    for char in text:
        if char == '<':
            stt = False
        if(stt):
            final = final + char
        if char == '>':
            stt = True
            final = final + ' '
    return final

df['text'] = df['review'].apply(remove_tags)

In [6]:
df['label'] = [1*(sent == 'positive') for sent in df['sentiment']]

In [7]:
from datasets import Dataset

dataset = Dataset.from_pandas(df)

In [8]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('roberta-base')

In [9]:
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [10]:
train_range = [i for i in range(0,5000) if 6>=i%10 and i%10>=0]
eval_range = [i for i in range(0,5000) if 9>=i%10 and i%10>=7]
train_dataset = tokenized_datasets.select(train_range).shuffle(seed=42)
eval_dataset = tokenized_datasets.select(eval_range) .shuffle(seed=42)

In [11]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("roberta-base", num_labels=2)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.bias', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classi

In [12]:

from transformers import TrainingArguments

training_args = TrainingArguments("test_trainer")

In [13]:
from transformers import Trainer

trainer = Trainer(
    model=model, args=training_args, train_dataset=train_dataset, eval_dataset=eval_dataset
)

In [14]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text, sentiment, review.
***** Running training *****
  Num examples = 3500
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1314


Step,Training Loss
500,0.372600
1000,0.205300


Saving model checkpoint to test_trainer/checkpoint-500
Configuration saved in test_trainer/checkpoint-500/config.json
Model weights saved in test_trainer/checkpoint-500/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-1000
Configuration saved in test_trainer/checkpoint-1000/config.json
Model weights saved in test_trainer/checkpoint-1000/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=1314, training_loss=0.2438101688657903, metrics={'train_runtime': 1128.6066, 'train_samples_per_second': 9.304, 'train_steps_per_second': 1.164, 'total_flos': 4020619115520000.0, 'train_loss': 0.2438101688657903, 'epoch': 3.0})

In [15]:

import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [16]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text, sentiment, review.
***** Running Evaluation *****
  Num examples = 1500
  Batch size = 8


{'eval_accuracy': 0.9186666666666666,
 'eval_loss': 0.42370206117630005,
 'eval_runtime': 52.8604,
 'eval_samples_per_second': 28.377,
 'eval_steps_per_second': 3.557}